In [ ]:
import os
import xml.etree.ElementTree as ET
import random
from collections import defaultdict
import json

# Tiền xử lý và chia tập dữ liệu từ file XML - GramVar

- **Mục đích**:  
  - Đọc dữ liệu từ các file XML gốc (theo cấu trúc `*_full.xml`).  
  - Gom nhóm các ví dụ dựa trên toàn bộ argument có dạng `ARG-n`.  
  - Chia dữ liệu thành 3 tập: Train, Validation, Test.  
  - Lưu kết quả thành các file `.json` để phục vụ huấn luyện mô hình sau này.  

- **Quy trình**

  1. **Hàm `parse_and_group_examples`**  
     - Đọc nội dung XML và lấy các phần tử `<example>`.  
     - Chuẩn hóa nhãn argument thành dạng `ARG-n`.  
     - Gom nhóm ví dụ theo toàn bộ bộ `ARG-n` (sắp xếp theo số thứ tự để tránh trùng lặp không cần thiết).  
     - Trả về danh sách nhóm và tổng số ví dụ.  

In [ ]:
def parse_and_group_examples(xml_content, predicate_name):
    """
    Phân tích nội dung XML, chuẩn hóa nhãn argument thành ARG-n,
    rồi gom nhóm các ví dụ theo toàn bộ tập các ARG-n.
    """
    groups = defaultdict(list)  # Dùng dict để gom nhóm: key = bộ ARG, value = list ví dụ

    try:
        # Parse nội dung XML (loại bỏ khoảng trắng thừa ở đầu/cuối)
        root = ET.fromstring(xml_content.strip())
    except ET.ParseError as e:
        # Nếu lỗi XML, in ra và bỏ qua
        print(f"Lỗi khi phân tích XML: {e}")
        return [], 0

    # Tìm tất cả các phần tử <example> trong file
    examples_found = root.findall('.//example')

    for ex in examples_found:
        # Lấy phần tử <text> (nội dung câu)
        text_element = ex.find('text')
        if text_element is None or text_element.text is None:
            continue
        text = text_element.text

        # Lưu các argument dưới dạng dict: {ARG-n: "nội dung"}
        args = {}
        for arg in ex.findall('arg'):
            key = arg.get('n')  # Lấy thuộc tính 'n' của <arg> (số thứ tự)
            if key and key.isdigit():
                new_key = f"ARG-{key}"   # Chuẩn hóa: nếu là số thì thành "ARG-<số>"
            else:
                new_key = key            # Nếu không phải số, giữ nguyên
            args[new_key] = arg.text

        # 1. Lấy tất cả các key có dạng 'ARG-...'
        arg_keys = [k for k in args if k.startswith('ARG-')]

        # 2. Sắp xếp key theo số thứ tự để thống nhất thứ tự
        #    => tránh trường hợp ('a','b') khác ('b','a')
        sorted_arg_keys = sorted(arg_keys, key=lambda k: int(k.split('-')[1]))

        # 3. Tạo khóa gom nhóm từ *giá trị* của các ARG (theo thứ tự đã sắp xếp)
        group_key = tuple(args[k] for k in sorted_arg_keys)

        # Đóng gói dữ liệu của 1 ví dụ
        sample_data = {
            "text": text,
            "predicate": predicate_name,
            "arguments": args
        }

        # Thêm ví dụ này vào nhóm tương ứng (theo group_key)
        groups[group_key].append(sample_data)

    # Trả về: danh sách các nhóm (list of list) và tổng số ví dụ tìm thấy
    return list(groups.values()), len(examples_found)

  2. **Hàm `split_groups_train_val_test`**  
     - Nhận danh sách các nhóm và chia thành 3 tập theo tỷ lệ: Train (80%), Validation (10%), Test (10%).  
     - Việc chia theo nhóm đảm bảo các ví dụ cùng pattern argument không bị rơi vào tập khác nhau (tránh rò rỉ dữ liệu).  
     - Flatten danh sách nhóm thành danh sách mẫu riêng lẻ.

In [ ]:
def split_groups_train_val_test(groups, val_size=0.1, test_size=0.1):
    """
    Chia danh sách nhóm dữ liệu thành 3 phần: Train, Validation, Test.
    - Mỗi "group" là tập các ví dụ có cùng pattern argument (gom từ bước trước).
    - Việc chia theo nhóm giúp tránh rò rỉ dữ liệu giữa các tập.

    Tham số:
      - groups: list các nhóm (mỗi nhóm chứa nhiều sample_data)
      - val_size: tỷ lệ validation (mặc định = 10%)
      - test_size: tỷ lệ test (mặc định = 10%)

    Trả về:
      - train_set, val_set, test_set: list các sample (đã flatten từ nhóm)
    """

    # 1. Kiểm tra tổng val_size + test_size phải < 1
    if val_size + test_size >= 1.0:
        raise ValueError("Tổng của val_size và test_size phải nhỏ hơn 1.")

    # 2. Xáo trộn ngẫu nhiên thứ tự các nhóm (để chia dữ liệu công bằng hơn)
    random.shuffle(groups)

    n_groups = len(groups)  # Tổng số nhóm

    # 3. Tính chỉ số cắt để lấy phần test (phía cuối)
    test_split_index = int(n_groups * (1 - test_size))

    # 4. Tính tỷ lệ validation trong phần còn lại sau khi tách test
    #    (ví dụ: val=0.1, test=0.1 => val chiếm 0.1/0.9 ≈ 11% phần còn lại)
    val_proportion_in_remainder = val_size / (1 - test_size)
    val_split_index = int(test_split_index * (1 - val_proportion_in_remainder))

    # 5. Cắt dữ liệu thành các nhóm train/val/test
    train_groups = groups[:val_split_index]
    val_groups   = groups[val_split_index:test_split_index]
    test_groups  = groups[test_split_index:]

    # 6. Flatten (từ list nhóm -> list sample) để huấn luyện dễ hơn
    train_set = [sample for group in train_groups for sample in group]
    val_set   = [sample for group in val_groups   for sample in group]
    test_set  = [sample for group in test_groups  for sample in group]

    # 7. Trả về kết quả
    return train_set, val_set, test_set

 3. **Chương trình chính**  
     - Khai báo đường dẫn dataset gốc và đường dẫn lưu kết quả.  
     - Tạo thư mục `Train`, `Validation`, `Test` trong thư mục đích.  
     - Duyệt qua tất cả file `*_full.xml`.  
     - Với mỗi file:  
       - Sinh ra `predicate_id` từ tên file (vd: `lemma_3_full.xml → lemma.03`).  
       - Parse nội dung và gom nhóm dữ liệu.  
       - Chia dữ liệu thành Train/Val/Test.  
       - Lưu ra 3 file `.json` tương ứng.  


In [ ]:
# 1. Khai báo các đường dẫn gốc
# dataset_path: chứa các file XML đầu vào
# output_path : nơi lưu dữ liệu sau khi chia Train/Validation/Test
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/GramVar/'
output_path  = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_GramVar/'

# 2. Tự động tạo các thư mục con (nếu chưa có)
# Mỗi tập dữ liệu sẽ được lưu riêng vào thư mục tương ứng
os.makedirs(os.path.join(output_path, 'Train'),      exist_ok=True)
os.makedirs(os.path.join(output_path, 'Validation'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'Test'),       exist_ok=True)

# 3. Lặp qua tất cả các file trong thư mục nguồn
print(f"Bắt đầu xử lý các file trong: {dataset_path}\n")
for filename in os.listdir(dataset_path):
    # Chỉ xử lý các file có đuôi "_full.xml"
    if filename.endswith('_full.xml'):
        print(f"Đang xử lý file: {filename}")

        # 3.1. Lấy tên file gốc (bỏ "_full.xml")
        base_name = filename.replace('_full.xml', '')

        # 3.2. Sinh ra "predicate_id" từ tên file
        #  - Nếu tên file có dạng lemma_3_full.xml -> lemma.03
        #  - Nếu chỉ có lemma_full.xml -> lemma.01
        predicate_id = ''
        if '_' in base_name:
            parts = base_name.split('_')
            lemma = parts[0]
            version_num = int(parts[1])
            predicate_id = f"{lemma}.{version_num:02d}"
        else:
            lemma = base_name
            predicate_id = f"{lemma}.01"

        print(f"Tên file gốc: {base_name}  -->  Predicate ID: {predicate_id}")

        # 3.3. Đọc nội dung XML từ file
        full_input_path = os.path.join(dataset_path, filename)
        try:
            with open(full_input_path, 'r', encoding='utf-8') as f:
                xml_content = f.read()
        except FileNotFoundError:
            print(f"LỖI: Không tìm thấy file '{full_input_path}'. Bỏ qua.")
            continue

        # 3.4. Parse nội dung XML và gom nhóm theo toàn bộ ARG-n
        all_groups, total_examples = parse_and_group_examples(xml_content, predicate_id)
        if not all_groups:
             print("Không tìm thấy mẫu nào hợp lệ. Bỏ qua.")
             continue

        print(f"Tìm thấy {total_examples} mẫu, gom thành {len(all_groups)} nhóm.")

        # 3.5. Chia dữ liệu thành Train/Val/Test (theo tỷ lệ mặc định: 80/10/10)
        train_data, val_data, test_data = split_groups_train_val_test(
            all_groups, val_size=0.1, test_size=0.1
        )
        print(f"Chia thành công: {len(train_data)} Train, {len(val_data)} Val, {len(test_data)} Test.")

        # 3.6. Xác định đường dẫn lưu dữ liệu sau khi chia
        train_output_path      = os.path.join(output_path, 'Train',      f'{base_name}_train_set.json')
        validation_output_path = os.path.join(output_path, 'Validation', f'{base_name}_validation_set.json')
        test_output_path       = os.path.join(output_path, 'Test',       f'{base_name}_test_set.json')

        # 3.7. Ghi dữ liệu ra file JSON với format dễ đọc (indent=2)
        with open(train_output_path, 'w', encoding='utf-8') as f:
            json.dump(train_data, f, indent=2, ensure_ascii=False)

        with open(validation_output_path, 'w', encoding='utf-8') as f:
            json.dump(val_data, f, indent=2, ensure_ascii=False)

        with open(test_output_path, 'w', encoding='utf-8') as f:
            json.dump(test_data, f, indent=2, ensure_ascii=False)

        print(f"Lưu thành công 3 file cho '{base_name}'.\n")

# 4. Hoàn tất toàn bộ quá trình
print("Hoàn tất! Tất cả các file đã được xử lý.")


- **Kết quả**:  
  - Với mỗi file XML gốc, sinh ra 3 file JSON:  
    - `<tên_file>_train_set.json`  
    - `<tên_file>_validation_set.json`  
    - `<tên_file>_test_set.json`  
  - Mỗi file chứa dữ liệu đã tiền xử lý, sẵn sàng cho bước huấn luyện mô hình.

# Tiền xử lý và chia tập dữ liệu từ file XML - ParaVE

In [ ]:
# 1. Khai báo các đường dẫn gốc
# dataset_path: thư mục chứa dữ liệu XML đầu vào (ParaVE corpus)
# split_parave_path: thư mục đích để lưu dữ liệu đã chia thành Train/Val/Test
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/ParaVE/'
split_parave_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/Split_ParaVE/'

# 2. Tự động tạo các thư mục con nếu chúng chưa tồn tại
# Mỗi tập dữ liệu (Train, Validation, Test) sẽ được lưu ở một thư mục riêng
os.makedirs(os.path.join(split_parave_path, 'Train'), exist_ok=True)
os.makedirs(os.path.join(split_parave_path, 'Validation'), exist_ok=True)
os.makedirs(os.path.join(split_parave_path, 'Test'), exist_ok=True)

# 3. Lặp qua tất cả các file trong thư mục nguồn
print(f"Bắt đầu xử lý các file trong: {dataset_path}\n")
for filename in os.listdir(dataset_path):
    # Chỉ xử lý các file XML có hậu tố "_full.xml"
    if filename.endswith('_full.xml'):
        print(f"--- Đang xử lý file: {filename} ---")

        # Lấy tên file gốc (vd: "begin_1" từ "begin_1_full.xml")
        base_name = filename.replace('_full.xml', '')

        # Sinh predicate_id theo định dạng chuẩn (vd: "begin.01")
        predicate_id = ''
        if '_' in base_name:
            # Nếu tên file có số version -> ghép lại thành lemma.xx
            parts = base_name.split('_')
            lemma = parts[0]
            version_num = int(parts[1])
            predicate_id = f"{lemma}.{version_num:02d}"
        else:
            # Nếu không có version -> mặc định là .01
            lemma = base_name
            predicate_id = f"{lemma}.01"

        print(f"Tên file gốc: {base_name}  -->  Predicate ID: {predicate_id}")

        # Tạo đường dẫn đầy đủ cho file XML
        full_input_path = os.path.join(dataset_path, filename)

        # Đọc nội dung file XML
        try:
            with open(full_input_path, 'r', encoding='utf-8') as f:
                xml_content = f.read()
        except FileNotFoundError:
            # Nếu không tìm thấy file -> bỏ qua
            print(f"LỖI: Không tìm thấy file '{full_input_path}'. Bỏ qua.")
            continue

        # Gọi hàm phân tích và gom nhóm dữ liệu theo ARG-n
        all_groups, total_examples = parse_and_group_examples(xml_content, predicate_id)

        # Nếu không có nhóm nào hợp lệ thì bỏ qua
        if not all_groups:
             print("Không tìm thấy mẫu nào hợp lệ. Bỏ qua.")
             continue

        print(f"Tìm thấy {total_examples} mẫu, gom thành {len(all_groups)} nhóm.")

        # Chia dữ liệu đã gom nhóm thành Train/Val/Test (mặc định 80/10/10)
        train_data, val_data, test_data = split_groups_train_val_test(all_groups, val_size=0.1, test_size=0.1)
        print(f"Chia thành công: {len(train_data)} Train, {len(val_data)} Val, {len(test_data)} Test.")

        # Xác định đường dẫn để lưu file JSON kết quả
        train_output_path = os.path.join(split_parave_path, 'Train', f'{base_name}_train_set.json')
        validation_output_path = os.path.join(split_parave_path, 'Validation', f'{base_name}_validation_set.json')
        test_output_path = os.path.join(split_parave_path, 'Test', f'{base_name}_test_set.json')

        # Lưu dữ liệu đã chia ra 3 file JSON
        with open(train_output_path, 'w', encoding='utf-8') as f:
            json.dump(train_data, f, indent=2, ensure_ascii=False)
        with open(validation_output_path, 'w', encoding='utf-8') as f:
            json.dump(val_data, f, indent=2, ensure_ascii=False)
        with open(test_output_path, 'w', encoding='utf-8') as f:
            json.dump(test_data, f, indent=2, ensure_ascii=False)

        print(f"Lưu thành công 3 file cho '{base_name}'.\n")

# 4. Hoàn tất quá trình xử lý tất cả file trong ParaVE
print("Hoàn tất! Tất cả các file trong ParaVE đã được xử lý.")

Bắt đầu xử lý các file trong: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/ParaVE/

--- Đang xử lý file: catalyse_full.xml ---
    Tên file gốc: catalyse  -->  Predicate ID: catalyse.01
    Tìm thấy 55 mẫu, gom thành 38 nhóm.
    --> Chia thành công: 43 Train, 5 Val, 7 Test.
    Lưu thành công 3 file cho 'catalyse'.

--- Đang xử lý file: result_full.xml ---
    Tên file gốc: result  -->  Predicate ID: result.01
    Tìm thấy 275 mẫu, gom thành 145 nhóm.
    --> Chia thành công: 218 Train, 26 Val, 31 Test.
    Lưu thành công 3 file cho 'result'.

--- Đang xử lý file: develop_full.xml ---
    Tên file gốc: develop  -->  Predicate ID: develop.01
    Tìm thấy 12 mẫu, gom thành 10 nhóm.
    --> Chia thành công: 10 Train, 1 Val, 1 Test.
    Lưu thành công 3 file cho 'develop'.

--- Đang xử lý file: eliminate_full.xml ---
    Tên file gốc: eliminate  -->  Predicate ID: eliminate.01
    Tìm thấy 42 mẫu, gom thành 33 nhóm.
    --> Chia thành công: 30 Train, 7 

# Gộp dữ liệu từ hai corpus (GramVar & ParaVE)

- **Mục đích**:  
  - Hợp nhất dữ liệu đã được chia sẵn (Train, Validation, Test) từ hai corpus khác nhau.  
  - Tạo ra một bộ dữ liệu kết hợp để huấn luyện mô hình tổng quát hơn.  
  - Lưu dữ liệu đã gộp thành các file `.json` riêng biệt cho từng tập.  


- **Quy trình**

 1. **Khai báo đường dẫn**  
     - Định nghĩa `gramvar_path` và `parave_path` trỏ tới hai thư mục nguồn.  
     - Tạo thư mục mới `Combined_Dataset` để chứa kết quả.  

In [ ]:
# KHAI BÁO CÁC ĐƯỜNG DẪN
# base_path: thư mục gốc chứa toàn bộ corpus
# gramvar_path: dữ liệu đã chia Train/Val/Test của GramVar
# parave_path : dữ liệu đã chia Train/Val/Test của ParaVE
# combined_output_path: thư mục mới để lưu dữ liệu sau khi gộp
base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/'
gramvar_path = os.path.join(base_path, 'Split_GramVar')
parave_path = os.path.join(base_path, 'Split_ParaVE')
combined_output_path = os.path.join(base_path, 'Combined_Dataset')

# TỰ ĐỘNG TẠO THƯ MỤC OUTPUT
os.makedirs(combined_output_path, exist_ok=True)
print(f"Thư mục output sẽ được lưu tại: {combined_output_path}")

2. **Xử lý theo từng tập con (Train, Validation, Test)**  
     - Lần lượt duyệt qua 3 thư mục con: `Train`, `Validation`, `Test`.  
     - Với mỗi tập con:  
       - Đọc toàn bộ file `.json` từ GramVar và ParaVE.  
       - Load dữ liệu và nối vào một list chung `combined_data`.

In [ ]:
# TIẾN HÀNH GỘP DỮ LIỆU
# subfolders: danh sách các tập con cần gộp (Train, Test, Validation)
subfolders = ['Train', 'Test', 'Validation']

for folder_name in subfolders:
    print(f"\n Bắt đầu gộp thư mục: '{folder_name}'")

    # List để chứa dữ liệu gộp từ cả hai nguồn (GramVar + ParaVE)
    combined_data = []

    # Xác định các thư mục nguồn cần đọc (vd: GramVar/Train, ParaVE/Train)
    source_folders_to_process = [
        os.path.join(gramvar_path, folder_name),
        os.path.join(parave_path, folder_name)
    ]

    # Lặp qua từng thư mục nguồn
    for source_path in source_folders_to_process:
        print(f"Đang đọc từ: {source_path}")

        # Kiểm tra xem thư mục có tồn tại không
        if not os.path.isdir(source_path):
            print(f"CẢNH BÁO: Thư mục không tồn tại. Bỏ qua.")
            continue

        # Lặp qua tất cả các file JSON trong thư mục nguồn
        for filename in os.listdir(source_path):
            if filename.endswith('.json'):
                file_path = os.path.join(source_path, filename)
                try:
                    # Đọc dữ liệu từ file JSON
                    with open(file_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                        # Thêm dữ liệu vào list tổng
                        combined_data.extend(data)
                except Exception as e:
                    print(f"Lỗi khi đọc file {filename}: {e}")

 3. **Lưu dữ liệu đã gộp**  
     - Nếu có dữ liệu, ghi ra file `.json` mới (vd: `combined_train_set.json`).  
     - Lưu trong thư mục `Combined_Dataset`.  

In [ ]:
 # LƯU FILE KẾT QUẢ ĐÃ GỘP
    if combined_data:
        # Đặt tên file output theo định dạng (vd: combined_train_set.json)
        output_filename = f"combined_{folder_name.lower()}_set.json"
        full_output_path = os.path.join(combined_output_path, output_filename)

        # Ghi dữ liệu đã gộp ra file JSON mới
        with open(full_output_path, 'w', encoding='utf-8') as f:
            json.dump(combined_data, f, indent=2, ensure_ascii=False)

        print(f"Gộp thành công {len(combined_data)} mẫu.")
        print(f"Đã lưu vào file: {output_filename}")
    else:
        print("Không có dữ liệu để gộp.")

print("\n Hoàn tất! Tất cả các file đã được gộp và lưu.")

- **Kết quả**:  
  - Nhận được 3 file JSON:  
    - `combined_train_set.json`  
    - `combined_validation_set.json`  
    - `combined_test_set.json`  
  - Mỗi file chứa dữ liệu đã hợp nhất từ cả hai corpus, sẵn sàng cho bước huấn luyện tiếp theo.  